In [1]:
import flappy_multi
import random
import numpy as np

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


## Agent

In [2]:
class NeatFlappy :
    s = 2
    def __init__(self, genome) :
        """Construit les nodes à partir du genome"""
        self.nodes = []
        self.genome = genome
        for i in range(nodeCount) :
            self.nodes.append(Node(genome, i, self.nodes))
    def shouldJump(self, v) :
        self.s += 1
        self.s -= min(0.8,(abs(v[0] - v[1])/100)**1)
        v = np.array(v+[1])*np.array([0.008,0.008, 0.02, 1])
        for node in self.nodes :
            node.reset()
        for i,vi in enumerate(v) :
            self.nodes[i].fireValue(vi)
        o = self.nodes[len(v)].value
        return o > 0.9
    
    def passedPip(self) :
        self.s += 100
    
    def score(self) :
        return self.s

## Parameters

In [3]:
numberPerGeneration = 50

In [4]:
numberInputs = 4
numberOutputs = 1
numberIo = numberOutputs + numberInputs

In [5]:
newLinkProba = 0.1
newNodeProba = 0.01
stayDisabledProba = 0.90
weigthMutationProba = 0.8
randomValueProva = 0.1
interspeciesProba = 0.005

In [6]:
#Distance max entre deux membre d'une même espèce
speciesDelta = 8
#Contribution  a la distance correspondant à un node non commun a deux agent
deltaDisjoint = 2
minPerSpecies = 5

## NEAT

### Evolution

In [7]:
#Nombre de type de links entre nodes
genomeSize = 0
#Nombre de Node possible
nodeCount = numberIo
#A quoi correspondent les links
links = []

class Gene :
    """Code pour un link entre deux node"""
    inIndex = 0
    outIndex = 0
    w = 1
    enabled = True

#Les inovations trouvées lors de la dernière génération
innovations = dict()

class Genome :
    species = 0
    def __init__(self) :
        self.genes = []

In [8]:
def initialGeneration() :
    """Crée la première génération"""
    innovations.clear()
    generation = []
    for i in range(numberPerGeneration) :
        generation.append(Genome())
    return [NeatFlappy(g) for g in generation]

In [9]:
def makeNextGeneration(generation) :
    """Fait évoluer une génération qui a déjà joué au jeux"""
    species = defineSpecies(generation)
    nbOffsprings = evaluateSpecies(species)
    print("Nombre de descendant : " + str(sum(nbOffsprings)), nbOffsprings)
    genomes = sortedGenomes(species)
    bred = breedSpecies(genomes, nbOffsprings)
    print("Nombre d'enfants crées " + str(len(bred)))
    return [NeatFlappy(g) for g in bred]

In [10]:
def sortedGenomes(species) :
    """A partir d'un ensemble d'agents rend un ensembre de génome classé du meilleur au moins bon"""
    genomes = []
    bestScore = -100
    for i in range(len(species)) :
        sortedAgents = sorted(species[i], key = lambda a : - a.score())
        genomes.append([a.genome for a in sortedAgents])
        if sortedAgents[0].score() > bestScore :
            bestScore = sortedAgents[0].score()
    print("Meilleu score de la Génération : " + str(bestScore))
    return genomes

In [11]:
def breedSpecies(species, nbOffsprings) :
    """Fait se reproduire les membres d'une espèce"""
    offsprings = []
    bp = breedingProbabilities(species)
    for i in range(len(species)) :
        s = species[i]
        nb = nbOffsprings[i]
        probs = bp[i]
        if nb :
            offsprings.append(s[0])
        nb-=1
        for i in range(nb) :
            g1 = randomGenomeFromSpecies(s,probs)
            g2 = randomGenomeFromSpecies(s,probs)
            if random.random() > interspeciesProba :
                g2 = randomAgent(species, bp)
            offsprings.append(breed(g1,g2))
    return offsprings

In [12]:
def defineSpecies(agents) :
    """Découpe la population en espèces"""
    species = []
    print("Nombre d'agents " + str(len(agents)))
    while len(agents) :
        a = agents[0]
        s = []
        for a2 in agents :
            if dist(a.genome, a2.genome) < deltaDisjoint :
                s.append(a2)
        for newSpecimen in s :
            agents.remove(newSpecimen)
        if len(s):
            species.append(s)
        else :
            print("Erreur : espèce vide")
    print("Nombre d'espèces : " + str(len(species)))
    if len(species) > 60 :
        print("Trop d'espèce")
        print([len(s) for s in species])
    return species

In [13]:
def dist(genome1,genome2) :
    """Calcule la distance entre deux génomes"""
    d = 0
    if len(genome1.genes) != len(genome2.genes) :
        normalizeGenome(genome1)
        normalizeGenome(genome2)
    n = len(genome1.genes)
    for i in range(n) :
        g1,g2 = genome1.genes[i], genome2.genes[i]
        if (g1 == None) != (g2 == None) :
            d += deltaDisjoint
        if g1 != None and g2 != None :
            d+= (g1.w - g2.w)**2 / n
    return d

In [14]:
def breedingProbabilities(species) :
    """Calcul pour chaque espèce la probabilité qu'un agent se reproduise, 
    sachant qu'on selectionne un agent de son espèce"""
    bp = []
    for s in species :
        probs = [0.001**(i/len(s)) for i in range(len(s))]
        s = sum(probs)
        probs = [p/s for p in probs]
        bp.append(probs)
    return bp

In [15]:
def randomGenomeFromSpecies(s, probs) :
    """Choisit un génome aléatoire dans un espèce"""
    r = random.random()
    i = 0
    while(r > probs[i]) :
        r -= probs[i]
        i+= 1
    return s[i]

In [16]:
def randomAgent(species, bp) :
    """Choisit un génome aleatoire dans la population"""
    i = random.randint(0, len(species) -1)
    return randomGenomeFromSpecies(species[i], bp[i])

In [17]:
def eliminateTeams(ratings) :
    iMin = -1
    for i,r in enumerate(ratings) :
        if (iMin ==-1 or r < ratings[iMin]) and r != 0 :
            iMin = i
    if ratings[iMin]*numberPerGeneration < minPerSpecies  :
        ratings[iMin] = 0
        ratings /= np.sum(ratings)
        eliminateTeams(ratings)
        return ratings
    return ratings
    

def evaluateSpecies(species) :
    """Evalue les espèces, et donne à combine d'enfant elles ont droit"""
    ratings = []
    for s in species :
        n = len(s)
        r = 0
        for a in s :
            r += a.score()/n
        ratings.append(r)
    s = sum(ratings)
    ratings = [r/s for r in ratings]
    
    ratings = eliminateTeams(ratings)
    s = sum(ratings)    
    numberOffsprings = [int(np.round(r/s*numberPerGeneration)) for r in ratings]
        
    diff = numberPerGeneration - sum(numberOffsprings)
    i = 0
    while(diff > 0) :
        i+= 1
        if numberOffsprings[i] > 0 :
            diff -= 1
            numberOffsprings[i] += 1
    while(diff < 0) :
        i+= 1
        if numberOffsprings[i] > 0 :
            diff += 1
            numberOffsprings[i] -= 1
    
            
        
    numberOffsprings[i] += diff
    
    print("Nombre d'enfants :")
    print(numberOffsprings)
    
    return numberOffsprings

### Genome

In [18]:
def mutate(g) :
    prevSize = len(g.genes)
    """Mute un génome"""
    for gene in g.genes :
        if gene != None and not gene.enabled and random.random() > stayDisabledProba() :
            gene.enabled  = True
    if random.random() < weigthMutationProba :
        for gene in g.genes :
            if gene != None :
                if random.random() < randomValueProva :
                    gene.w = random.uniform(-1,1)
                else :
                    gene.w += random.uniform(-0.1,0.1)
    
    if random.random() < newLinkProba :
        makeNewLink(g)
    if random.random() < newNodeProba :
        makeNewNode(g)
        
    #print("taille Genome muté : " + str(len(g.genes)) + " Pour normalement : " + str(genomeSize) + " Avant : " + str(prevSize))
        

In [19]:
def makeNewLink(g) :
    """Crée une nouvelle conection sur entre deux nodes"""
    i = random.randint(0, nodeCount - 1)
    j = i
    iNodes = range(0,numberInputs)
    oNodes = range(numberInputs, numberIo)
    while j == i or (i in iNodes and j in iNodes) or (i in oNodes and j in oNodes)  :
        j = random.randint(0, nodeCount - 1)
    print(i,j)
    if j in iNodes :
        i,j = j,i
    if i in oNodes :
        i,j = j,i
    if i in findSuccessors(j) :
        i,j = j,i
    geneIndex = -1
    if (i,j) in innovations.keys() :
        geneIndex = innovations[(i,j)]
    else :
        #Si on a crér un nouveau link, il faut mettre a jour la variable comptant les links existant
        global genomeSize
        geneIndex = genomeSize
        innovations[(i,j)] = genomeSize
        genomeSize += 1
        links.append((i,j))
        print("Nouveau link : ")
        print((i,j),innovations[(i,j)])
    normalizeGenome(g)
    gene = Gene()
    gene.inIndex = i
    gene.outIndex = j
    gene.w = random.uniform(-1,1)
    g.genes[geneIndex] = gene
    
    

In [20]:
def findSuccessors(i) :
    """Regarde quels nodes suivent un node en descendant les links"""
    successors = []
    for link in links :
        a,b = link
        if a==i :
            successors.append(b)
            for s in findSuccessors(b) :
                if s not in successors :
                    successors.append(s)
    return successors

In [21]:
def makeNewNode(g) :
    """Crée un nouveau node en mutant un link si on en trouve un"""
    linksCandidate = []
    for gene in g.genes :
        if gene != None and gene.enabled :
            linksCandidate.append(gene)
    if not len(linksCandidate) :
        return
    gene = random.choice(linksCandidate)
    geneIndex = g.genes.index(gene)
    i,j = -1,-1
    nodeIndex = -1
    if geneIndex in innovations.keys() :
        i,j,nodeIndex = innovations[geneIndex]
    else : 
        global nodeCount
        global genomeSize
        nodeIndex = nodeCount
        nodeCount += 1
        i,j = genomeSize, genomeSize+1
        genomeSize += 2
        print("Nouveau node : ")
        print(str(nodeIndex) + " devient : " + str((i,j)) + " Nouveau node " + str(nodeIndex))
    normalizeGenome(g)
    gene1, gene2 = Gene(), Gene()
    gene1.inIndex = gene.inIndex
    gene1.outIndex = nodeIndex
    gene2.inIndex = nodeIndex
    gene2.outIndex = gene.outIndex
    gene2.w = gene.w
    

In [22]:
def breed(g1, g2) :
    """Croise deux génomes"""
    g = Genome()
    g.species = g1.species
    normalizeGenome(g1)
    normalizeGenome(g2)
    #print("Init Size : " + str(len(g.genes)))
    for i in range(genomeSize) :
        if random.random() > 0.5 :
            g.genes.append(g1.genes[i])
        else :
            g.genes.append(g2.genes[i])
    #print("before mutaition Size : " + str(len(g.genes)))
    #print("genomeSize : " + str(genomeSize))
    mutate(g)
    return g
        

In [23]:
def normalizeGenome(g) :
    """Fait en sorte que la taille d'un génome soit la bonne quitte à rajouter des None"""
    missing = max(0,genomeSize-len(g.genes))
    #print("missing : " + str(missing))
    g.genes += [None for i in range(missing)]
    #if len(g.genes) != genomeSize :
     #   print("erreur")
      #  print(len(g.genes), genomeSize)

### Network

In [24]:
def activationFunction(x) :
    return 1/(1+np.exp(-x))

class Node :
    """Implémentation d'un réseau de neurones à partir des nodes
    Attributs :
    predecessors : le nombre de nodes en imput
    remaining : le nombre de node en input qui doivent encore fire
    successors : la liste des nodes output
    nodes : la liste de tous les nodes
    value : l'activation du node"""
    predecessors = 0
    remaining = 0
    successors = []
    nodes = []
    value = 0
    def __init__(self, g, index, nodes):
        for gene in g.genes :
            if gene != None and gene.inIndex == index and gene.enabled:
                self.successors.append((gene.outIndex, gene.w))
            if gene != None and gene.outIndex == index and gene.enabled :
                self.predecessors += 1
        self.nodes = nodes
        self.reset()
    def fireValue(self, value) :
        for (i,w) in self.successors :
            if nodeCount <= i :
                print("Erreur : i trop grand pour nodeCount ",i,nodeCount)
            if nodeCount != len(self.nodes) :
                print("nodes trop petit pour nodeCount", len(self.nodes), nodeCount)
            self.nodes[i].receiveValue(value*w)
    def receiveValue(self, v) :
        self.value += v
        self.remaining -= 1
        if self.remaining == 0 :
            self.value = activationFunction(self.value)
            self.fireValue(self.value)
    
    def reset(self) :
        self.remaining = self.predecessors
        self.value = 0

## Fight

In [25]:
genomeSize = 0
nodeCount = numberIo
genNumber = 0
generation = initialGeneration()
#generation = makeNextGeneration(generation)

In [26]:
nextAgentIndex = 0
def nextNeatAgent() :
    global nextAgentIndex
    global generation
    global genNumber
    if nextAgentIndex == numberPerGeneration :
        generation = makeNextGeneration(generation)
        genNumber += 1
        nextAgentIndex = 0
        print("Génération " + str(genNumber))
    agent = generation[nextAgentIndex]
    #describe(agent)
    nextAgentIndex += 1
    return agent

In [27]:
def describe(agent):
    print("Agent :")
    for gene in agent.genome.genes :
        if gene!= None and gene.enabled :
            print(gene.inIndex, gene.outIndex, gene.w)

In [ ]:
flappy_multi.main(nextNeatAgent)

Nombre d'agents 50
Nombre d'espèces : 1
Nombre d'enfants :
[50]
Nombre de descendant : 50 [50]
Meilleu score de la Génération : 22.339999999999996
3 4
Nouveau link : 
(3, 4) 0
0 4
Nouveau link : 
(0, 4) 1
0 4
0 4
2 4
Nouveau link : 
(2, 4) 2
2 4
Nombre d'enfants crées 50
Génération 1
Nombre d'agents 50
Nombre d'espèces : 4
Nombre d'enfants :
[12, 13, 12, 13]
Nombre de descendant : 50 [12, 13, 12, 13]
Meilleu score de la Génération : 30.199999999999942
1 4
Nouveau link : 
(1, 4) 3
1 4
3 4
3 4
0 4
1 4
1 4
3 4
Nombre d'enfants crées 50
Génération 2
Nombre d'agents 50
Nombre d'espèces : 9
Nombre d'enfants :
[5, 5, 4, 6, 6, 6, 6, 6, 6]
Nombre de descendant : 50 [5, 5, 4, 6, 6, 6, 6, 6, 6]
Meilleu score de la Génération : 30.199999999999953
4 0
0 4
Nouveau node : 
5 devient : (4, 5) Nouveau node 5
3 5
Nouveau link : 
(3, 5) 6
Nombre d'enfants crées 50
Génération 3
Nombre d'agents 50
Nombre d'espèces : 12
Nombre d'enfants :
[6, 5, 6, 6, 0, 6, 5, 6, 5, 0, 0, 5]
Nombre de descendant : 50 [6, 5,

## Test

In [ ]:
activationFunction(1)